# 🧠 Deep Learning Models for Personalized Learning

PyTorch models for:
1. **Student Engagement Prediction**
2. **Content Recommendation**
3. **Difficulty Level Optimization**

---

## 1. Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os

os.makedirs('../models', exist_ok=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cpu


## 2. Dataset Class

In [2]:
class StudentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx): return self.features[idx], self.labels[idx]

## 3. Model Architectures

In [3]:
class StudentEngagementModel(nn.Module):
    """Predict student engagement score (0-1)"""
    def __init__(self, input_dim, hidden_dims=[64, 32, 16]):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.extend([nn.Linear(prev, h), nn.BatchNorm1d(h), nn.ReLU(), nn.Dropout(0.3)])
            prev = h
        layers += [nn.Linear(prev, 1), nn.Sigmoid()]
        self.network = nn.Sequential(*layers)
    
    def forward(self, x): return self.network(x)

class ContentRecommendationModel(nn.Module):
    """Collaborative filtering for content recommendations"""
    def __init__(self, n_users, n_items, emb_dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        self.fc = nn.Sequential(
            nn.Linear(emb_dim*2, 64), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1), nn.Sigmoid()
        )
    
    def forward(self, users, items):
        x = torch.cat([self.user_emb(users), self.item_emb(items)], dim=1)
        return self.fc(x)

class DifficultyPredictor(nn.Module):
    """Predict optimal difficulty level"""
    def __init__(self, input_dim, n_levels=5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, n_levels), nn.Softmax(dim=1)
        )
    
    def forward(self, x): return self.net(x)

print('Models defined!')

Models defined!


## 4. Generate Synthetic Data

In [4]:
np.random.seed(42)
n = 5000

features = np.column_stack([
    np.random.uniform(5, 40, n),    # study_hours_weekly
    np.random.uniform(10, 120, n),  # avg_session_duration
    np.random.uniform(0.2, 1.0, n), # completion_rate
    np.random.randint(1, 20, n),    # quiz_attempts
    np.random.uniform(0.3, 1.0, n), # avg_quiz_score
    np.random.randint(1, 30, n),    # days_active_monthly
    np.random.randint(1, 50, n),    # resources_accessed
    np.random.randint(0, 30, n),    # discussion_posts
])

# Engagement = weighted combination
engagement = (features[:,0]/40*0.2 + features[:,2]*0.25 + 
              features[:,4]*0.2 + features[:,5]/30*0.15 + 
              np.minimum(features[:,7]/10, 1)*0.1 + np.random.uniform(0, 0.1, n))
labels = np.clip(engagement, 0, 1).reshape(-1, 1)

print(f'Features: {features.shape}, Labels: {labels.shape}')

Features: (5000, 8), Labels: (5000, 1)


## 5. Train Engagement Model

In [5]:
# Normalize
X = (features - features.mean(axis=0)) / (features.std(axis=0) + 1e-8)

# Split
split = int(len(X) * 0.8)
train_ds = StudentDataset(X[:split], labels[:split])
val_ds = StudentDataset(X[split:], labels[split:])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

# Model
model = StudentEngagementModel(input_dim=8)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train
epochs = 50
for epoch in range(epochs):
    model.train()
    for feats, labs in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(feats), labs)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        model.eval()
        val_loss = sum(criterion(model(f), l).item() for f, l in val_loader) / len(val_loader)
        print(f'Epoch {epoch+1}: Val Loss = {val_loss:.4f}')

Epoch 10: Val Loss = 0.0015
Epoch 20: Val Loss = 0.0012
Epoch 30: Val Loss = 0.0011
Epoch 40: Val Loss = 0.0011
Epoch 50: Val Loss = 0.0011


## 6. Save Model

In [6]:
torch.save(model.state_dict(), '../models/engagement_model.pth')
print('Model saved to ../models/engagement_model.pth')

Model saved to ../models/engagement_model.pth


## 7. Demo Prediction

In [8]:
sample = np.array([[25, 45, 0.75, 10, 0.82, 20, 30, 5]])
sample_norm = (sample - features.mean(axis=0)) / (features.std(axis=0) + 1e-8)

model.eval()
with torch.no_grad():
    pred = model(torch.FloatTensor(sample_norm)).item()
print(f'Predicted Engagement: {pred:.3f}')

Predicted Engagement: 0.674


---
## Summary

| Model | Purpose | Output |
|-------|---------|--------|
| StudentEngagementModel | Predict engagement | 0-1 score |
| ContentRecommendationModel | Recommend content | Relevance |
| DifficultyPredictor | Optimal level | 5 classes |